In [ ]:
CLIENT_ID=...
CLIENT_SECRET=...
TENANT_ID=...

Example Use

In [ ]:
from graph_helper import GraphHelper

GraphHelper.initialize_graph_for_user_auth()
user_info = GraphHelper.get_user()  # May fail in app-only context
messages = GraphHelper.get("/users/youruser@yourdomain.com/mailFolders/inbox/messages?$top=5")

print(messages)

In [ ]:
# graph_helper.py
# Replacement for GraphHelper.php in PHP
# Uses Microsoft MSAL for app-only authentication

import os
import msal
import requests

class GraphHelper:
    """
    GraphHelper handles token acquisition and authenticated requests
    using Microsoft Graph API with app-only credentials.
    """

    # Static class variables for token and Graph endpoint
    token = None
    graph_endpoint = "https://graph.microsoft.com/v1.0"

    @staticmethod
    def initialize_graph_for_user_auth():
        """
        Authenticate using client credentials and acquire a token.
        Equivalent to GraphHelper::initializeGraphForUserAuth() in PHP.
        """
        client_id = os.getenv("CLIENT_ID")
        client_secret = os.getenv("CLIENT_SECRET")
        tenant_id = os.getenv("TENANT_ID")

        if not all([client_id, client_secret, tenant_id]):
            raise ValueError("Missing one or more required environment variables.")

        authority = f"https://login.microsoftonline.com/{tenant_id}"
        app = msal.ConfidentialClientApplication(
            client_id=client_id,
            client_credential=client_secret,
            authority=authority
        )

        # Acquire token for app
        result = app.acquire_token_silent(scopes=["https://graph.microsoft.com/.default"], account=None)
        if not result:
            result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])

        if "access_token" not in result:
            raise Exception(f"Could not acquire token: {result.get('error_description')}")

        GraphHelper.token = result["access_token"]

    @staticmethod
    def get_user():
        """
        Equivalent to GraphHelper::getUser() in PHP.
        Fetches information about the service principal's identity.
        """
        if GraphHelper.token is None:
            raise Exception("Graph not initialized. Call initialize_graph_for_user_auth first.")

        headers = {
            "Authorization": f"Bearer {GraphHelper.token}"
        }

        # This endpoint only works for delegated auth — app-only accounts may not have a user object.
        response = requests.get(f"{GraphHelper.graph_endpoint}/me", headers=headers)

        if response.status_code != 200:
            raise Exception(f"Graph API call failed: {response.text}")

        return response.json()

    @staticmethod
    def get(path: str):
        """
        Perform a GET request to any Microsoft Graph endpoint path.
        E.g., /users/{user}/mailFolders/inbox/messages
        """
        if GraphHelper.token is None:
            raise Exception("Graph not initialized. Call initialize_graph_for_user_auth first.")

        url = f"{GraphHelper.graph_endpoint}{path}"
        headers = {
            "Authorization": f"Bearer {GraphHelper.token}"
        }

        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()


START

SET timezone to America/New_York

LOAD environment variables:
    - CLIENT_ID
    - CLIENT_SECRET
    - TENANT_ID
    - AUTH_TENANT
    - GRAPH_USER_SCOPES

SET pdf_dir = "/home/portal/batch/pdfupload004"
DEFINE list of valid US state codes

CONNECT to SQL Server database (host = atfsdb.jws.com, db = atfs)

IF file named 'FAILED' does NOT exist:
    CALL readToken()        // prepares local token cache
    CALL initializeGraph()  // sets up Graph client
    IF greetUser() returns true:
        CALL updateToken()      // saves token updates
        CALL processAff004Email()  // main processing logic

FUNCTION initializeGraph():
    - Call GraphHelper.initializeGraphForUserAuth()

FUNCTION greetUser():
    TRY
        - Call GraphHelper.getUser()
        - If success, return True
    CATCH error:
        - Log error with timestamp
        - Write 'FAILED' file with error
        - Send email to support@jws.com
        - Return False

FUNCTION readToken():
    - (not fully shown yet)

END


In [ ]:
# process_aff004_email.py (partial)

import os
import datetime
from graph_helper import GraphHelper
import smtplib
from email.mime.text import MIMEText

FAILED_FILE = "FAILED"
PDF_DIR = "/home/portal/batch/pdfupload004"
US_STATES = {"AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VI", "VT", "WA", "WI", "WV", "WY"}

# Set timezone for logs
os.environ['TZ'] = 'America/New_York'

# STEP 1: Check if FAILED file exists
if not os.path.exists(FAILED_FILE):
    # STEP 2: Read cached token if needed (placeholder)
    def read_token():
        pass  # your token logic here

    # STEP 3: Initialize Graph
    def initialize_graph():
        GraphHelper.initialize_graph_for_user_auth()

    # STEP 4: Try to get user and handle errors
    def greet_user():
        try:
            user = GraphHelper.get_user()
            return True
        except Exception as e:
            timestamp = datetime.datetime.now().isoformat()
            error_msg = f"{timestamp} Error getting user: {str(e)}\n"
            print(error_msg)

            # Write FAILED file
            with open(FAILED_FILE, "w") as f:
                f.write(error_msg)

            # Send notification
            send_email("support@jws.com",
                       "Inspection PDF Processing - Token expired",
                       "<b>Office 365 token for AFF_004 inspection pdf processing has expired.</b><br><b>Or other error has occurred.</b><br><b>Check the FAILED file in /home/portal/batch/pdfupload004 folder.</b>",
                       "Office 365 token for AFF_004 inspection pdf processing has expired.\n\n")
            return False

    # STEP 5: Email sender
    def send_email(to, subject, html_body, plain_body):
        msg = MIMEText(plain_body, 'plain')
        msg['Subject'] = subject
        msg['From'] = "noreply@yourdomain.com"
        msg['To'] = to

        with smtplib.SMTP('localhost') as server:
            server.send_message(msg)

    read_token()
    initialize_graph()
    if greet_user():
        # update_token() → to be implemented
        # process_aff004_email() → to be implemented
        pass
